# Aufgabe 5: Separierbarkeit der Gauß-Filterung
Als effizientere Variante kann eine zweidimensionale lineare Filterung auf zwei eindimensionale lineare Filterungen reduziert werden.

Ein linearer zweidimensionaler Filter $A \in \mathbb{R}^{m \times n}$ heißt separierbar, wenn er durch Faltung zweier eindimensionaler Filter dargestellt werden kann:
\begin{align}
 A &= D_1 * D_2, \qquad \text{mit $D_1 \in \mathbb{R}^m, D_2 \in \mathbb{R}^n$}\\
                &= D_1 \cdot D_2^\top,  \qquad \text{(da $D_1,D_2$ Vektoren).}
\end{align}
Somit ergibt sich die Faltung zu
\begin{align}
  I * A &= I * (D_1 * D_2)\\
        &= (I * D_1) * D_2 \qquad \text{(Assoziativität der Faltung)}
\end{align}


Implementieren Sie nun die 2-D-Gaußfilterung als Hintereinanderausführung je eines Gaußfilters in vertikaler und horizontaler Richtung!

Vergleichen Sie die Laufzeiten mit der nicht-separierten Filterung für verschiedene Größen der Filtermaske!

## 0. Pfade, Pakete etc.

In [ ]:
import glob
import urllib.request

%matplotlib inline
import matplotlib.pyplot as plt

import imageio
import numpy as np

In [ ]:
image_filter = 'Bilder/*.jpg'

## 1. Definition der Faltungsmaske
Definieren Sie hier wie in der vorherigen Aufgabe zunächst die Parameter `m` und `sigma` des Filters. Berechnen Sie anschließend eine eindimensionale Filtermaske `A_gauss`!

In [ ]:
m = 7
sigma = m / 5

A_gauss = np.array([[np.exp(-(np.divide(k**2, 2*(sigma**2))))] for k in range(m)])

## 2. Laden des Bildes

In [ ]:
image_path = np.random.choice(glob.glob(image_filter))
image = imageio.imread(image_path)

Für diese Aufgabe ist es wichtig, das Bild im Fließkommaformat vorliegen zu haben. Konvertieren sie `image` zu einer geeigneten Repräsentation:

In [ ]:
image = np.asarray(image, dtype=np.float32) / 255

## 3. Berechung der Faltung
Setzen Sie hier die Funktion `ex2_convolve` aus der vorherigen Aufgabe ein:

In [ ]:
def ex2_convolve(image, filter_mask):
    convolved_image = np.zeros_like(image)

    mask_h, mask_k = tuple(value // 2 for value in filter_mask.shape)
    image_y, image_x = image.shape

    for y in range(image_y):
        for x in range(image_x):
            sum = 0
            for h in range(-mask_h, mask_h+1):
                for k in range(-mask_k, mask_k+1):
                    if 0<=y-h and y-h<=image_y-1 and 0<=x-k and x-k<=image_x-1:
                        sum = sum + filter_mask[h,k]*image[y-h,x-k]

            convolved_image[y,x] = np.divide(1, (2 * np.pi * (sigma**2))) * sum
    
    return convolved_image

## 4. Separierter Gauß-Filter

Berechnen Sie nun das gefaltete Bild durch zwei Aufrufe der obigen Funktion! Tipp: Verwenden Sie die Funktion `transpose` aus dem Paket `numpy`, um die Filtermaske zu transponieren.

In [ ]:
%%time
convolved_image = ex2_convolve(ex2_convolve(image, A_gauss), A_gauss.transpose())

## 5. Darstellung
Um die Wirksamkeit des separierten Gauß-Filters zu überprüfen, stellen Sie `image` und `convolved_image` nebeneinander dar:

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 12))
axes[0].set_title('Image Orginal')
axes[0].imshow(image, cmap='gray')
axes[1].set_title('Image Convolved')
axes[1].imshow(convolved_image, cmap='gray')
plt.show()

## 6. Laufzeitvergleich für verschiedene Filtergrößen

Vergleichen Sie die Laufzeit der separierten Filterung mit der der 2D-Filterung für verschieden große Filtermasken!

Zur Zeitmessung können Sie die magische Jupyter-Funktion `%time` (pro Zeile) oder `%%time` (pro Zelle) verwenden.

In [ ]:
m = np.random.choice([3, 5, 7, 10, 20])
print('m = {}'.format(m))
sigma = m / 5

A_gauss_2d = np.array([[(np.exp(-(np.divide(h**2 + k**2, 2*(sigma**2))))) for h in range(m)] for k in range(m)])
A_gauss_1d = np.array([[np.exp(-(np.divide(k**2, 2*(sigma**2))))] for k in range(m)])

%time convolved_image_2d = ex2_convolve(image, A_gauss_2d)
%time convolved_image_1d = ex2_convolve(ex2_convolve(image, A_gauss_1d), A_gauss.transpose())